# MEV-CBE Power Roll Up Example

Link to the example in the Cookbook:\
[https://mms.openmbee.org/alfresco/mmsapp/mms.html#/projects/PROJECT-ID_2[...]8/views/_19_0_2_8b50284_1582591660624_573606_56409](https://mms.openmbee.org/alfresco/mmsapp/mms.html#/projects/PROJECT-ID_2_2_17_7_33_25_AM_3ccfaf88_159fe0d7ba9__7d4e_cae_tw_jpl_nasa_gov_127_0_0_1/master/documents/_18_0_6_88f0277_1497548395207_75642_302748/views/_19_0_2_8b50284_1582591660624_573606_56409)


## Structure & Behavior
![img/MEV_structure.svg](img/MEV_structure.svg)
![img/Comp1.jpg](img/Comp1.jpg)
## Analysis
![img/MEV_analysis.svg](img/MEV_analysis.svg)
## Constraint
![img/MEV_constraint.svg](img/MEV_constraint.svg)


In [2]:
package 'MEV-CBE Power Roll Up'{
    import SI::Real;
    import ScalarFunctions::sum;
    import 'Power Roll-Up Pattern'::*;
    import Behavior::*;
    import Structure::*;
    
    value type Power specializes Real;
    
    package 'Power Roll-Up Pattern' {
        block 'PowerRollUpPattern'{
            part subPower : 'PowerRollUpPattern'[*];
            
            value 'totalPower' : Power = sum(subPower::totalPower) + prBehavior::power;
            
            exhibit prBehavior : 'Behavior::Simple Power Behavior';

            state def 'Simple Power Behavior'{
                value 'power' : Power;
            }

        } //END block 'Data Roll-up Aspect'
    } // END package 'Power Roll-Up Pattern'

    
    package Behavior {
        value type e1;
        value type e2;
        
        state def 'Component Behavior' specializes 'Power Roll-Up Pattern::Simple Power Behavior' {
            entry; then S1;
           
            state S1{
                // assert unbound power == s1Power; // if using a solver
                entry action{
                    power = s1Power; 
                }
            }

            transition S1_to_S2
                first S1
                accept e1 then S2;
            
            state S2{
                entry action{
                    power = s2Power; 
                }
            }
            
            transition S2_to_S1
                first S2
                accept e2 then S1;
        } // END state def 'Component Behavior'
    } // END package Behavior
    
    package 'Structure' {
        block Car{
            part comp1 {
                exhibit c1Behavior : 'Component Behavior';
                value s1Power : Power = 1;
                value s2Power : Power = 2;
            }

            part comp2 {
                exhibit c2Behavior : 'Component Behavior';
                value s1Power : Power = 3;
                value s2Power : Power = 4;
          }
        }       
    }
    
    package Configurations {
        // how to specify different configs?
    }
    
    package 'Analysis' {
        block 'Car CBE' specializes Car, 'Power Roll-Up Pattern'::'PowerRollUpPattern'{
            value totalPower = Power Roll-Up Pattern::totalPower;
            
            part redefines comp1 :> 'Power Roll-Up Pattern'::'PowerRollUpPattern' subsets subPower {
                // having two exhibited behaviors now. Both have a power VP
                // need to bind power from roll-up pattern behavior to power in component behavior
                // prBehavior::power = c1Behavior::power; // possible?
                // or redefine both inherited behaviors
                // exhibit behavior:'Component Behavior' redefines prBehavior, c1Behavior; // is that possible?
                value redefines s1Power = 1;
                value redefines s2Power = 2;
            }
            part redefines comp2 :> 'Power Roll-Up Pattern'::'PowerRollUpPattern' subsets subPower {
                value redefines s1Power = 7;
                value redefines s2Power = 4;
            }
        }
        
        block 'Car MEV' specializes Car, 'Power Roll-Up Pattern'::'PowerRollUpPattern'{
           value totalPower = Power Roll-Up Pattern::totalPower;

            part redefines comp1 :> 'Power Roll-Up Pattern'::'PowerRollUpPattern' subsets subPower {
                value redefines s1Power = 1;
                value redefines s2Power = 2;
            }
            part redefines comp2 :> 'Power Roll-Up Pattern'::'PowerRollUpPattern' subsets subPower {
                value redefines s1Power = 3;
                value redefines s2Power = 4;
            }
        }
 
        block Analysis {
            value margin : Real = carMEV::totalPower - carCBE::totalPower;
            
            part carCBE : 'Car CBE';
            part carMEV : 'Car MEV';
            
            perform action {
                send e1 to carCBE::comp1;
                send e1 to carCBE::comp2;
                send e1 to carMEV::comp1;
                send e1 to carMEV::comp2;
            }
        }
    } // END package 'Analysis'
    
}

ERROR:mismatched input ';' expecting 'as' (2.sysml line : 16 column : 56)
ERROR:mismatched input ';' expecting 'as' (2.sysml line : 58 column : 55)
ERROR:mismatched input ';' expecting 'as' (2.sysml line : 62 column : 55)
ERROR:no viable alternative at input 'Roll' (2.sysml line : 83 column : 32)
ERROR:no viable alternative at input 'Pattern' (2.sysml line : 83 column : 43)
ERROR:no viable alternative at input 'Roll' (2.sysml line : 96 column : 31)
ERROR:no viable alternative at input 'Pattern' (2.sysml line : 96 column : 42)
